### Assistente de Apostilas (pesquisa em documentos PDFs e responde com contexto restrito)

In [30]:
#import openai
from openai import OpenAI
#print(openai.__version__)

In [31]:
#client = openai.Client()   # método antigo (deprecated)
client = OpenAI()

In [32]:
#vector_store = client.beta.vector_stores.create(name="Tutor de Apostilas")  # versão anterior (depracated)
vector_store = client.vector_stores.create(name="Tutor de Apostilas")
print(vector_store.id)

vs_68ab64eee3488191aa5223bd0d05ca90


In [33]:
# Pode passar vários arquivos, mas vamos passar, apenas, um arquivo
file = ["files/LLM.pdf"]
file_stream = [open(f, "rb") for f in file]
#file_batch = client.beta.vector_stores.file_batches.upload_and_poll(        # versão anterior (depracated)
file_batch = client.vector_stores.file_batches.upload_and_poll(
    vector_store_id = vector_store.id,
    files = file_stream)

In [34]:
file_batch.status

'completed'

In [35]:
file_batch.file_counts

FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)

In [36]:
assistant = client.beta.assistants.create(
    name="Tutor Apostila",
    instructions="Você é um tutor especializado em tecnologias emergentes. Você sabe responder perguntas sobre LLMs como OpenAI, HuggingFace, etc. Caso você não encontre as respostas, seja sincero e fale que não sabe responder",
    tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
    model="gpt-4-turbo-preview")

In [46]:
#pergunta = "Conforme o documento, o que é o Hugging Face?"
pergunta = "Conforme o documento, o que é o OpenAI?"

In [47]:
# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = pergunta)

C:\Users\Naja\AppData\Local\Temp\ipykernel_2348\3798620811.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\Naja\AppData\Local\Temp\ipykernel_2348\3798620811.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


In [48]:
# Executa a thread
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
    instructions = "Nome de usuário premium")

C:\Users\Naja\AppData\Local\Temp\ipykernel_2348\3617845786.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


In [49]:
# Aguarda o thread rodar
import time 
while run.status in ["queued","in_progress","cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id)

C:\Users\Naja\AppData\Local\Temp\ipykernel_2348\953612562.py:5: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


In [50]:
# Verificar a resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id = thread.id
    )

    print(mensagens)
else:
    print(f"Erro: {run.status}")

C:\Users\Naja\AppData\Local\Temp\ipykernel_2348\1930001157.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  mensagens = client.beta.threads.messages.list(


SyncCursorPage[Message](data=[Message(id='msg_29Ewg2s37ihoaoFSNIOS864S', assistant_id='asst_D17gPCbaL1CyMmxOiCdRFSYN', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=694, file_citation=FileCitation(file_id='file-D8oezt7qUE5x3YMuTGgCgc'), start_index=681, text='【4:0†LLM.pdf】', type='file_citation')], value='A OpenAI é descrita no documento fornecido como uma organização que lançou o GPT-3 em 2020, o qual se tornou o maior modelo de linguagem com 175 bilhões de parâmetros, estabelecendo um novo referencial de desempenho para tarefas relacionadas à linguagem. Em 2022, a OpenAI lançou o ChatGPT, transformando o GPT-3 e modelos semelhantes em um serviço amplamente acessível aos usuários através de uma interface web. Isso iniciou um aumento significativo na conscientização pública sobre LLMs (Large Language Models) e IA generativa. A OpenAI também esteve envolvida no lançamento do GPT-4 em 2023, que estabeleceu novos refer

In [51]:
print(mensagens.data[0].content[0].text.value)

A OpenAI é descrita no documento fornecido como uma organização que lançou o GPT-3 em 2020, o qual se tornou o maior modelo de linguagem com 175 bilhões de parâmetros, estabelecendo um novo referencial de desempenho para tarefas relacionadas à linguagem. Em 2022, a OpenAI lançou o ChatGPT, transformando o GPT-3 e modelos semelhantes em um serviço amplamente acessível aos usuários através de uma interface web. Isso iniciou um aumento significativo na conscientização pública sobre LLMs (Large Language Models) e IA generativa. A OpenAI também esteve envolvida no lançamento do GPT-4 em 2023, que estabeleceu novos referenciais tanto em tamanho de parâmetros quanto em desempenho【4:0†LLM.pdf】.


In [52]:
# Analisando os passos do modelo
run_steps = client.beta.threads.runs.steps.list(
    thread_id = thread.id,
    run_id = run.id)

C:\Users\Naja\AppData\Local\Temp\ipykernel_2348\764390232.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_steps = client.beta.threads.runs.steps.list(


In [53]:
for step in run_steps.data[::-1]:
    print(f"\n===Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" * 10)
            if tool_call.type == "code_interpreter":
                print("Code Interpreter input:")
                print(tool_call.code_interpreter.input)
            elif tool_call.type == "file_search":
                print("File Search query:")
                print(tool_call.file_search)
            else:
                print(f"Tool call de tipo {tool_call.type} não tratado")
            print("=" * 10)

    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id,
        )
        if message.content[0].type == "text":
            print(message.content[0].text.value)
        if message.content[0].type == "image_file":
            file_id = message.content[0].image_file.file_id
            image_data = client.files.content(file_id)
            with open(f"files/{file_id}.png", "wb") as f:
                f.write(image_data.read())
                print(f"Imagem: {file_id} foi salva.")



===Step tool_calls
File Search query:
FileSearch(ranking_options=FileSearchRankingOptions(ranker='default_2024_08_21', score_threshold=0.0), results=[FileSearchResult(file_id='file-D8oezt7qUE5x3YMuTGgCgc', file_name='LLM.pdf', score=0.716421568903066, content=None, attributes={}), FileSearchResult(file_id='file-D8oezt7qUE5x3YMuTGgCgc', file_name='LLM.pdf', score=0.6666366860941911, content=None, attributes={}), FileSearchResult(file_id='file-D8oezt7qUE5x3YMuTGgCgc', file_name='LLM.pdf', score=0.4739663451082819, content=None, attributes={}), FileSearchResult(file_id='file-D8oezt7qUE5x3YMuTGgCgc', file_name='LLM.pdf', score=0.413633825409534, content=None, attributes={}), FileSearchResult(file_id='file-D8oezt7qUE5x3YMuTGgCgc', file_name='LLM.pdf', score=0.39315253753892354, content=None, attributes={}), FileSearchResult(file_id='file-D8oezt7qUE5x3YMuTGgCgc', file_name='LLM.pdf', score=0.3248121636458754, content=None, attributes={}), FileSearchResult(file_id='file-D8oezt7qUE5x3YMuTGgCg

C:\Users\Naja\AppData\Local\Temp\ipykernel_2348\3912564020.py:17: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.retrieve(


A OpenAI é descrita no documento fornecido como uma organização que lançou o GPT-3 em 2020, o qual se tornou o maior modelo de linguagem com 175 bilhões de parâmetros, estabelecendo um novo referencial de desempenho para tarefas relacionadas à linguagem. Em 2022, a OpenAI lançou o ChatGPT, transformando o GPT-3 e modelos semelhantes em um serviço amplamente acessível aos usuários através de uma interface web. Isso iniciou um aumento significativo na conscientização pública sobre LLMs (Large Language Models) e IA generativa. A OpenAI também esteve envolvida no lançamento do GPT-4 em 2023, que estabeleceu novos referenciais tanto em tamanho de parâmetros quanto em desempenho【4:0†LLM.pdf】.
